In [ ]:
from datacube import Datacube
import numpy as np
import xarray as xr
import dask
from datacube.utils.dask import start_local_dask
import matplotlib.pyplot as plt
import sys

sys.path.append('../dea-notebooks/Scripts')
from dea_datahandling import load_ard
from dea_dask import create_local_dask_cluster

In [ ]:
create_local_dask_cluster()

In [ ]:
lat, lon = -33.2, 149.1
buffer = 0.05
time = ('1987', '2010')

In [ ]:
dc = Datacube(env='c3-samples')

query = {'lon': (lon - buffer, lon + buffer),
         'lat': (lat - buffer, lat + buffer),
         'time': time,
         'measurements':['nbart_red', 'nbart_nir'],
         'output_crs' :'EPSG:3577',
         'resolution' : (-30, 30),
         'resampling' :{"fmask": "nearest", "*": "average"}}

In [ ]:
ds = load_ard(dc=dc, 
              dask_chunks={'x':150, 'y':150},
              products=['ga_ls5t_ard_3', 'ga_ls7e_ard_3'],
              ls7_slc_off=False,
              **query)
print(ds)

In [ ]:
quarter= {'JFM': [1,2,3],
                       'FMA': [2,3,4],
                       'MAM': [3,4,5],
                       'AMJ': [4,5,6],
                       'MJJ': [5,6,7],
                       'JJA': [6,7,8],
                       'JAS': [7,8,9],
                       'ASO': [8,9,10],
                       'SON': [9,10,11],
                       'OND': [10,11,12],
                       'NDJ': [11,12,1],
                       'DJF': [12,1,2],
                      }

def ndvi_clim_mean(data):
    
    def attrs_reassign(da, dtype=np.float32):
        da_attr = data.attrs
        da = da.assign_attrs(**da_attr)
        return da
    
    ndvi = xr.Dataset(data_vars={'ndvi': (data.nbart_nir - data.nbart_red) / (data.nbart_nir + data.nbart_red)},
                          coords=data.coords,
                          attrs=dict(crs=data.crs))
    
    ndvi_var = []
    for q in quarter:
        ix=ndvi['time.month'].isin(quarter[q])
        ndvi_clim_mean=ndvi.where(ix,drop = True).std(dim='time')   
        ndvi_clim_mean=ndvi_clim_mean.to_array(name=q).drop('variable').squeeze()
        ndvi_var.append(ndvi_clim_mean)

    q_clim_mean = xr.merge(ndvi_var)   

    #assign back attributes
    q_clim_mean.attrs = data.attrs 
    q_clim_mean = q_clim_mean.apply(attrs_reassign, keep_attrs=True)  

    return q_clim_mean

In [ ]:
clim = ndvi_clim_mean(ds)
print(clim)